In [3]:
"""
============================================================================
ЯЧЕЙКА 1: ИМПОРТЫ И НАСТРОЙКИ
============================================================================
Тестирование OCR моделей с параллельной обработкой через ThreadPoolExecutor
"""

# ============================================================================
# СТАНДАРТНЫЕ БИБЛИОТЕКИ
# ============================================================================
import os
import sys
import json
import base64
import logging
import time
import csv
from pathlib import Path
from io import BytesIO
from datetime import datetime
from typing import List, Dict, Optional, Tuple, Any
from dataclasses import dataclass, field, asdict

# ============================================================================
# ПАРАЛЛЕЛЬНАЯ ОБРАБОТКА
# ============================================================================
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

# ============================================================================
# ВНЕШНИЕ БИБЛИОТЕКИ
# ============================================================================
from PIL import Image
import requests
from tqdm import tqdm
from openai import OpenAI

# ============================================================================
# ВИЗУАЛИЗАЦИЯ И АНАЛИЗ
# ============================================================================
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ============================================================================
# НАСТРОЙКА ЛОГИРОВАНИЯ
# ============================================================================
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler('ocr_processing.log', encoding='utf-8')
    ]
)
logger = logging.getLogger(__name__)

# ============================================================================
# ГЛОБАЛЬНЫЕ ПЕРЕМЕННЫЕ
# ============================================================================
# Добавить корень проекта в PYTHONPATH
PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name != "Team_5_AI_Tutor":
    PROJECT_ROOT = PROJECT_ROOT.parents[1]
sys.path.insert(0, str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / "data"
IMAGES_DIR = PROJECT_ROOT / "notebooks" / "vlm-ingestion-pipeline" / "test_images"
OUTPUT_DIR = DATA_DIR / "test_results"

# Создаём директории если не существуют
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Настройка визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")


print("✅ Ячейка 1: Импорты загружены")
print(f"   📂 Корневая директория: {PROJECT_ROOT}")
print(f"   📂 Директория с изображениями: {IMAGES_DIR}")
print(f"   📂 Директория для результатов: {OUTPUT_DIR}")


✅ Ячейка 1: Импорты загружены
   📂 Корневая директория: /home/gna/workspase/education/MEPHI/Team_5_AI_Tutor
   📂 Директория с изображениями: /home/gna/workspase/education/MEPHI/Team_5_AI_Tutor/notebooks/vlm-ingestion-pipeline/test_images
   📂 Директория для результатов: /home/gna/workspase/education/MEPHI/Team_5_AI_Tutor/data/test_results


In [4]:
"""
============================================================================
ЯЧЕЙКА 2: СТРУКТУРЫ ДАННЫХ
============================================================================
"""

@dataclass
class OCRResult:
    """
    Результат OCR обработки одного изображения.
    
    Thread-safety: ✅ Immutable после создания
    """
    model_name: str           # Название модели (HunyuanOCR, Qwen3-VL, и т.д.)
    folder_name: str          # Название папки/книги
    filename: str             # Имя файла (page_001.png)
    page_number: int          # Номер страницы
    text: str                 # Распознанный текст
    tokens: int               # Количество токенов
    processing_time: float    # Время обработки (секунды)
    text_length: int          # Длина текста (символов)
    timestamp: str            # ISO timestamp
    error: Optional[str] = None  # Текст ошибки (если есть)
    
    def __post_init__(self):
        """Автозаполнение полей"""
        if not self.timestamp:
            self.timestamp = datetime.now().isoformat()
        if self.text_length == 0:
            self.text_length = len(self.text)


@dataclass
class ModelStats:
    """
    Статистика работы модели.
    
    Thread-safety: ✅ Создаётся после завершения всех потоков
    """
    model_name: str
    total_images: int
    folders_processed: int
    total_time: float
    avg_time_per_image: float
    total_tokens: int
    total_chars: int
    success_rate: float
    errors: int
    throughput: float  # images per second


@dataclass
class FolderStats:
    """
    Статистика по одной папке/книге.
    
    Thread-safety: ✅ Создаётся после завершения всех потоков
    """
    folder_name: str
    total_images: int
    total_time: float
    avg_time_per_image: float
    total_chars: int
    avg_chars_per_image: float
    success_rate: float


# Глобальное хранилище результатов
ALL_RESULTS = {}      # {model_name: [OCRResult, ...]}
FOLDER_GROUPS = {}    # {folder_name: [image_paths, ...]}
IMAGE_PATHS = []      # Список всех путей к изображениям

print("✅ Ячейка 2: Структуры данных определены")
print("   - OCRResult: результат обработки одного изображения")
print("   - ModelStats: статистика модели")
print("   - FolderStats: статистика по папке")


✅ Ячейка 2: Структуры данных определены
   - OCRResult: результат обработки одного изображения
   - ModelStats: статистика модели
   - FolderStats: статистика по папке


In [5]:
"""
============================================================================
ЯЧЕЙКА 3: КОНФИГУРАЦИЯ (ИСПРАВЛЕНО ИМЯ МОДЕЛИ)
============================================================================
"""

MODELS_CONFIG = {
    "olmOCR-2-7B-1025": {
        "enabled": True,
        "port": 1234,
        "model_path": "allenai/olmocr-2-7b",  # ← ИСПРАВЛЕНО! Точное имя из LM Studio
        
        # Параметры подготовки изображений
        "target_longest_dim": 512,     # Размер изображения
        "jpeg_quality": 85,            # Качество JPEG
        
        # Параметры запроса
        "max_tokens": 4096,            # Или -1 для авто
        "temperature": 0.0,            # Детерминизм для OCR
        
        # Retry логика
        "max_retries": 3,
        "retry_scale_factor": 0.8,
        
        "prompts": {
            "default": "",
        }
    },
    "qwen3-vl-4b": {
        "enabled": True,
        "port": 1234,
        "model_path": "allenai/olmocr-2-7b",  # ← ИСПРАВЛЕНО! Точное имя из LM Studio
        
        # Параметры подготовки изображений
        "target_longest_dim": 512,     # Размер изображения
        "jpeg_quality": 85,            # Качество JPEG
        
        # Параметры запроса
        "max_tokens": 4096,            # Или -1 для авто
        "temperature": 0.0,            # Детерминизм для OCR
        
        # Retry логика
        "max_retries": 3,
        "retry_scale_factor": 0.8,
        
        "prompts": {
            "default": "",
        }
    }
}

PROMPT_TYPE = "default"

print("✅ Ячейка 3: Конфигурация модели")
print(f"   Модель: {MODELS_CONFIG['olmOCR-2-7B-1025']['model_path']}")
print(f"   Порт: {MODELS_CONFIG['olmOCR-2-7B-1025']['port']}")


✅ Ячейка 3: Конфигурация модели
   Модель: allenai/olmocr-2-7b
   Порт: 1234


In [6]:
"""
============================================================================
ЯЧЕЙКА 4: ЗАГРУЗКА ИЗОБРАЖЕНИЙ
============================================================================
"""

def load_images_from_directory(images_dir: Path) -> Tuple[List[Path], Dict[str, List[Path]]]:
    """
    Загружает изображения из директории, группируя по папкам.
    
    Returns:
        (image_paths, folder_groups)
    """
    image_paths = []
    folder_groups = {}
    
    # Поддерживаемые форматы
    image_extensions = {'.png', '.jpg', '.jpeg', '.webp', '.bmp'}
    
    # Рекурсивный обход
    for folder in sorted(images_dir.iterdir()):
        if not folder.is_dir():
            continue
        
        folder_name = folder.name
        folder_images = []
        
        for img_file in sorted(folder.iterdir()):
            if img_file.suffix.lower() in image_extensions:
                image_paths.append(img_file)
                folder_images.append(img_file)
        
        if folder_images:
            folder_groups[folder_name] = folder_images
            print(f"  📂 {folder_name}: {len(folder_images)} изображений")
    
    return image_paths, folder_groups


# Загрузка
print("🔍 Поиск изображений...")
IMAGE_PATHS, FOLDER_GROUPS = load_images_from_directory(IMAGES_DIR)

print(f"\n✅ Ячейка 4: Изображения загружены")
print(f"   Всего изображений: {len(IMAGE_PATHS)}")
print(f"   Папок: {len(FOLDER_GROUPS)}")


🔍 Поиск изображений...
  📂 o-predelnom-mnogomernom-raspredelenii: 4 изображений
  📂 standartizatsiya-i-kachestvo-zhizni: 3 изображений
  📂 Книга_Python_К_вершинам_мастерства_Рамальо_Лучано: 5 изображений

✅ Ячейка 4: Изображения загружены
   Всего изображений: 12
   Папок: 3


In [7]:
"""
============================================================================
ЯЧЕЙКА 5: ФУНКЦИИ СОХРАНЕНИЯ РЕЗУЛЬТАТОВ
============================================================================
"""

def save_results_to_files(
    results: List[OCRResult],
    output_dir: Path,
    model_name: str
) -> Dict[str, Path]:
    """
    Сохраняет результаты в различных форматах.
    
    Thread-safety: ✅ Вызывается после завершения всех потоков
    """
    saved_files = {}
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Группировка по папкам
    folder_groups = {}
    for result in results:
        if result.folder_name not in folder_groups:
            folder_groups[result.folder_name] = []
        folder_groups[result.folder_name].append(result)
    
    # 1. MARKDOWN файлы по папкам
    for folder_name, folder_results in folder_groups.items():
        md_path = output_dir / f"{model_name}_{folder_name}_{timestamp}.md"
        
        with open(md_path, 'w', encoding='utf-8') as f:
            f.write(f"# OCR Результаты: {folder_name}\n\n")
            f.write(f"Модель: {model_name}\n")
            f.write(f"Дата: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"{'='*80}\n\n")
            
            for result in sorted(folder_results, key=lambda r: r.page_number):
                f.write(f"\n{'─'*80}\n")
                f.write(f"СТРАНИЦА {result.page_number}\n")
                f.write(f"Файл: {result.filename}\n")
                f.write(f"Время: {result.processing_time:.2f}с\n")
                f.write(f"Символов: {result.text_length}\n")
                
                if result.error:
                    f.write(f"❌ ОШИБКА: {result.error}\n")
                else:
                    f.write(f"{'─'*80}\n\n")
                    f.write(result.text)
                    f.write(f"\n\n")
        
        saved_files[f"md_{folder_name}"] = md_path
    
    # 2. JSON файл (полные результаты)
    json_path = output_dir / f"{model_name}_results_{timestamp}.json"
    json_data = [asdict(r) for r in results]
    
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=2)
    
    saved_files['json'] = json_path
    
    # 3. CSV файл (статистика)
    csv_path = output_dir / f"{model_name}_stats_{timestamp}.csv"
    
    with open(csv_path, 'w', encoding='utf-8', newline='') as f:
        fieldnames = [
            'folder_name', 'filename', 'page_number', 'text_length',
            'tokens', 'processing_time', 'error'
        ]
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        
        for result in results:
            writer.writerow({
                'folder_name': result.folder_name,
                'filename': result.filename,
                'page_number': result.page_number,
                'text_length': result.text_length,
                'tokens': result.tokens,
                'processing_time': result.processing_time,
                'error': result.error or ''
            })
    
    saved_files['csv'] = csv_path
    
    return saved_files


print("✅ Ячейка 5: Функции сохранения определены")


✅ Ячейка 5: Функции сохранения определены


In [8]:
"""
============================================================================
ЯЧЕЙКА 6: ФУНКЦИИ СТАТИСТИКИ
============================================================================
"""

def calculate_model_stats(results: List[OCRResult]) -> ModelStats:
    """
    Вычисляет статистику для модели.
    
    Thread-safety: ✅ Вызывается после завершения всех потоков
    """
    if not results:
        return ModelStats(
            model_name="Unknown",
            total_images=0,
            folders_processed=0,
            total_time=0,
            avg_time_per_image=0,
            total_tokens=0,
            total_chars=0,
            success_rate=0,
            errors=0,
            throughput=0
        )
    
    model_name = results[0].model_name
    total_time = sum(r.processing_time for r in results)
    total_tokens = sum(r.tokens for r in results)
    total_chars = sum(r.text_length for r in results)
    
    # Успешные результаты (без ошибок)
    success_count = sum(1 for r in results if not r.error)
    success_rate = (success_count / len(results)) * 100 if results else 0
    
    # Количество ошибок
    errors = sum(1 for r in results if r.error)
    
    # Папки
    folders = set(r.folder_name for r in results)
    
    return ModelStats(
        model_name=model_name,
        total_images=len(results),
        folders_processed=len(folders),
        total_time=total_time,
        avg_time_per_image=total_time / len(results) if results else 0,
        total_tokens=total_tokens,
        total_chars=total_chars,
        success_rate=success_rate,
        errors=errors,
        throughput=len(results) / total_time if total_time > 0 else 0
    )


def calculate_folder_stats(results: List[OCRResult], folder_name: str) -> Optional[FolderStats]:
    """
    Вычисляет статистику для одной папки.
    
    Thread-safety: ✅ Вызывается после завершения потоков
    """
    folder_results = [r for r in results if r.folder_name == folder_name]
    
    if not folder_results:
        return None
    
    total_time = sum(r.processing_time for r in folder_results)
    total_chars = sum(r.text_length for r in folder_results)
    
    # Успешные результаты
    success_count = sum(1 for r in folder_results if not r.error)
    success_rate = (success_count / len(folder_results)) * 100 if folder_results else 0
    
    return FolderStats(
        folder_name=folder_name,
        total_images=len(folder_results),
        total_time=total_time,
        avg_time_per_image=total_time / len(folder_results) if folder_results else 0,
        total_chars=total_chars,
        avg_chars_per_image=total_chars / len(folder_results) if folder_results else 0,
        success_rate=success_rate
    )


print("✅ Ячейка 6: Функции статистики определены")


✅ Ячейка 6: Функции статистики определены


In [9]:
"""
============================================================================
ЯЧЕЙКА 7: ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ (СТИЛЬ olmOCR)
============================================================================
"""

import base64
from io import BytesIO
from PIL import Image
import requests
from typing import Optional

def image_to_base64(
    image: Image.Image, 
    target_longest_dim: int = 512,
    jpeg_quality: int = 85
) -> str:
    """
    Конвертирует PIL Image в base64 по методике Allen Institute olmOCR.
    
    Оптимизировано для LM Studio с ограниченным контекстом.
    
    Args:
        image: PIL изображение
        target_longest_dim: Максимальный размер по длинной стороне (512-672px)
        jpeg_quality: Качество JPEG сжатия (70-95, рекомендуется 85)
    
    Returns:
        Base64-строка
    """
    # 1. Resize по длинной стороне (метод olmOCR)
    if max(image.size) > target_longest_dim:
        scale = target_longest_dim / max(image.size)
        new_size = (int(image.width * scale), int(image.height * scale))
        image = image.resize(new_size, Image.Resampling.LANCZOS)
    
    # 2. Конвертация в RGB
    if image.mode != 'RGB':
        # Для RGBA/LA делаем белый фон (важно для формул)
        if image.mode in ('RGBA', 'LA'):
            background = Image.new('RGB', image.size, (255, 255, 255))
            if image.mode == 'RGBA':
                background.paste(image, mask=image.split()[3])
            else:
                background.paste(image, mask=image.split()[1])
            image = background
        else:
            image = image.convert('RGB')
    
    # 3. JPEG сжатие
    buffered = BytesIO()
    image.save(
        buffered,
        format='JPEG',
        quality=jpeg_quality,
        optimize=True,
        progressive=True
    )
    
    img_bytes = buffered.getvalue()
    
    # 4. Логирование размера (опционально)
    size_kb = len(img_bytes) / 1024
    if size_kb > 150:
        logger.warning(f"Изображение {size_kb:.1f}KB может быть большим для контекста")
    
    return base64.b64encode(img_bytes).decode()


def check_model_availability(port: int, timeout: int = 5) -> bool:
    """
    Проверяет доступность LM Studio сервера.
    
    Args:
        port: Порт LM Studio
        timeout: Таймаут подключения
    
    Returns:
        True если сервер доступен
    """
    try:
        response = requests.get(
            f"http://localhost:{port}/v1/models",
            timeout=timeout
        )
        return response.status_code == 200
    except Exception as e:
        logger.error(f"Ошибка проверки порта {port}: {e}")
        return False


print("✅ Ячейка 7: Вспомогательные функции определены")
print("   - image_to_base64() - оптимизация изображений (стиль olmOCR)")
print("   - check_model_availability() - проверка LM Studio")


✅ Ячейка 7: Вспомогательные функции определены
   - image_to_base64() - оптимизация изображений (стиль olmOCR)
   - check_model_availability() - проверка LM Studio


In [10]:
"""
============================================================================
ЯЧЕЙКА 8: КЛИЕНТ ДЛЯ LM STUDIO (БЕЗ ЗАПУСКА СЕРВЕРА)
============================================================================
"""

from openai import OpenAI
from PIL import Image
import time
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from typing import Dict, Any, Tuple, Optional, List
from pathlib import Path


def ocr_single_image(
    img_path: Path,
    folder_name: str,
    model_name: str,
    model_config: Dict[str, Any],
    prompt_type: str,
    page_num: int
) -> OCRResult:
    """
    Отправляет изображение в УЖЕ ЗАПУЩЕННЫЙ LM Studio.
    
    НЕ запускает сервер - только отправляет запросы!
    """
    start_time = time.time()
    
    try:
        # 1. Загрузка и подготовка изображения
        image = Image.open(img_path)
        original_size = image.size
        
        # 2. Конвертация в base64 с сжатием
        target_dim = model_config.get("target_longest_dim", 512)
        jpeg_quality = model_config.get("jpeg_quality", 85)
        
        image_base64 = image_to_base64(
            image, 
            target_longest_dim=target_dim,
            jpeg_quality=jpeg_quality
        )
        
        base64_kb = len(image_base64) / 1024
        
        # 3. Создание клиента (подключение к УЖЕ ЗАПУЩЕННОМУ LM Studio)
        client = OpenAI(
            base_url=f"http://localhost:{model_config['port']}/v1",
            api_key="lm-studio",  # Любое значение для LM Studio
            timeout=180
        )
        
        # 4. Формирование запроса
        messages = [
            {
                "role": "user",
                "content": f"data:image/jpeg;base64,{image_base64}"
            }
        ]
        
        # 5. Отправка запроса в LM Studio
        # ТОЛЬКО параметры генерации, НЕ параметры модели!
        response = client.chat.completions.create(
            # model=model_config["model_path"],
            model="qwen3-vl-4b",
            messages=messages,
            temperature=model_config.get("temperature", 0.0),
            max_tokens=model_config.get("max_tokens", 4096)
        )
        
        # 6. Извлечение результата
        text = response.choices[0].message.content
        tokens = response.usage.total_tokens if response.usage else 0
        processing_time = time.time() - start_time
        
        logger.info(
            f"✅ {img_path.name}: {original_size} → {target_dim}px → "
            f"{base64_kb:.1f}KB → {tokens:,} tok → "
            f"{len(text):,} симв за {processing_time:.2f}с"
        )
        
        return OCRResult(
            model_name=model_name,
            folder_name=folder_name,
            filename=img_path.name,
            page_number=page_num,
            text=text,
            tokens=tokens,
            processing_time=processing_time,
            text_length=len(text),
            timestamp=datetime.now().isoformat(),
            error=None
        )
        
    except Exception as e:
        processing_time = time.time() - start_time
        error_msg = str(e)
        logger.error(f"❌ {img_path.name}: {error_msg}")
        
        return OCRResult(
            model_name=model_name,
            folder_name=folder_name,
            filename=img_path.name,
            page_number=page_num,
            text="",
            tokens=0,
            processing_time=processing_time,
            text_length=0,
            timestamp=datetime.now().isoformat(),
            error=error_msg
        )


def ocr_with_retry(
    img_path: Path,
    folder_name: str,
    model_name: str,
    model_config: Dict[str, Any],
    prompt_type: str,
    page_num: int
) -> OCRResult:
    """
    Retry логика с автоматическим уменьшением размера изображения.
    """
    max_retries = model_config.get("max_retries", 3)
    retry_scale = model_config.get("retry_scale_factor", 0.8)
    
    config_copy = model_config.copy()
    
    for attempt in range(max_retries):
        result = ocr_single_image(
            img_path, folder_name, model_name, 
            config_copy, prompt_type, page_num
        )
        
        if not result.error:
            if attempt > 0:
                logger.info(f"✅ {img_path.name}: Успех на попытке {attempt + 1}")
            return result
        
        # Ошибка контекста - уменьшаем изображение
        if "context" in result.error.lower() or "tokens" in result.error.lower():
            if attempt < max_retries - 1:
                new_dim = int(config_copy["target_longest_dim"] * retry_scale)
                logger.warning(
                    f"⚠️ {img_path.name}: Retry {attempt + 1}/{max_retries}, "
                    f"уменьшаю размер {config_copy['target_longest_dim']}px → {new_dim}px"
                )
                config_copy["target_longest_dim"] = new_dim
                time.sleep(1)
                continue
        
        if attempt < max_retries - 1:
            logger.warning(f"⚠️ {img_path.name}: Retry {attempt + 1}/{max_retries}")
            time.sleep(1)
    
    return result


def ocr_single_image_wrapper(args: Tuple) -> OCRResult:
    """Wrapper для ThreadPoolExecutor"""
    return ocr_with_retry(*args)


def test_model_parallel(
    model_name: str,
    max_workers: int = 4
) -> Optional[List[OCRResult]]:
    """
    Параллельная отправка запросов в LM Studio.
    
    НЕ запускает сервер - только использует УЖЕ ЗАПУЩЕННЫЙ!
    """
    
    # Валидация
    if 'IMAGE_PATHS' not in globals() or not IMAGE_PATHS:
        print(f"❌ Изображения не загружены! Запустите Ячейку 4")
        return None
    
    if model_name not in MODELS_CONFIG:
        print(f"❌ Модель {model_name} не найдена в MODELS_CONFIG")
        return None
    
    model_config = MODELS_CONFIG[model_name]
    
    if not model_config["enabled"]:
        print(f"⏸️  {model_name} отключена")
        return None
    
    # Проверка доступности LM Studio
    if not check_model_availability(model_config["port"]):
        print(f"❌ LM Studio недоступен на порту {model_config['port']}")
        print(f"   💡 Убедитесь что:")
        print(f"      1. LM Studio запущен")
        print(f"      2. Модель загружена (кнопка 'Load Model')")
        print(f"      3. Сервер запущен (вкладка 'Local Server')")
        return None
    
    # Заголовок
    print(f"{'='*70}")
    print(f"🚀 ТЕСТИРОВАНИЕ: {model_name}")
    print(f"{'='*70}")
    print(f"✅ LM Studio доступен на порту {model_config['port']}")
    print(f"📊 Изображений: {len(IMAGE_PATHS)}")
    print(f"⚡ Параллельных потоков: {max_workers}")
    print(f"🖼️  Размер изображений: {model_config['target_longest_dim']}px")
    print(f"🔄 Retry: {model_config.get('max_retries', 3)} попытки")
    print(f"{'='*70}\n")
    
    # Подготовка задач
    tasks = []
    for folder_name, folder_paths in FOLDER_GROUPS.items():
        for page_num, img_path in enumerate(folder_paths, 1):
            task = (
                img_path, folder_name, model_name,
                model_config, PROMPT_TYPE, page_num
            )
            tasks.append(task)
    
    print(f"📦 Задач: {len(tasks)}")
    print(f"   ~{len(tasks) / max_workers:.1f} задач на worker\n")
    
    # Параллельная обработка
    start_time = time.time()
    results = []
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_task = {
            executor.submit(ocr_single_image_wrapper, task): task
            for task in tasks
        }
        
        with tqdm(total=len(tasks), desc=f"⚡ {model_name}", unit="img") as pbar:
            for future in as_completed(future_to_task):
                try:
                    result = future.result()
                    results.append(result)
                    
                    status = "✅" if not result.error else "❌"
                    pbar.set_postfix({
                        'файл': result.filename[:18],
                        'время': f"{result.processing_time:.1f}с",
                        'токены': f"{result.tokens}",
                        'символов': result.text_length,
                        'статус': status
                    })
                    
                except Exception as exc:
                    logger.error(f"Worker exception: {exc}")
                    img_path = future_to_task[future][0]
                    results.append(OCRResult(
                        model_name=model_name,
                        folder_name=future_to_task[future][1],
                        filename=img_path.name,
                        page_number=future_to_task[future][5],
                        text="",
                        tokens=0,
                        processing_time=0,
                        text_length=0,
                        timestamp=datetime.now().isoformat(),
                        error=f"Worker exception: {exc}"
                    ))
                
                pbar.update(1)
    
    total_time = time.time() - start_time
    
    # Сохранение
    ALL_RESULTS[model_name] = results
    print(f"\n💾 Сохранение результатов...")
    saved_files = save_results_to_files(results, OUTPUT_DIR, model_name)
    
    # Статистика
    stats = calculate_model_stats(results)
    
    print(f"\n{'='*70}")
    print(f"📊 СТАТИСТИКА {model_name}")
    print(f"{'='*70}")
    print(f"  Изображений:      {stats.total_images}")
    print(f"  Папок:            {stats.folders_processed}")
    print(f"  Общее время:      {stats.total_time:.1f}с ({stats.total_time/60:.1f} мин)")
    print(f"  Throughput:       {stats.throughput:.2f} изобр/с")
    print(f"  Среднее время:    {stats.avg_time_per_image:.2f}с/изобр")
    print(f"  Токенов:          {stats.total_tokens:,}")
    print(f"  Символов:         {stats.total_chars:,}")
    print(f"  Успешно:          {stats.success_rate:.1f}%")
    
    if stats.errors > 0:
        print(f"  ⚠️  Ошибок:       {stats.errors}")
    
    if len(FOLDER_GROUPS) > 1:
        print(f"\n📂 По папкам:")
        for folder_name in FOLDER_GROUPS.keys():
            folder_stats = calculate_folder_stats(results, folder_name)
            if folder_stats:
                print(f"  {folder_name}: {folder_stats.total_images} изобр, "
                      f"{folder_stats.avg_time_per_image:.2f}с, "
                      f"{folder_stats.avg_chars_per_image:.0f} симв")
    
    print(f"\n💾 Файлы: {len(saved_files)}")
    for filepath in saved_files.values():
        print(f"  📄 {filepath.name}")
    
    print(f"\n{'='*70}")
    print(f"✅ ЗАВЕРШЕНО!")
    print(f"{'='*70}\n")
    
    return results


def test_model(model_name: str, max_workers: int = None):
    """Алиас с автовыбором workers"""
    if max_workers is None:
        if 'IMAGE_PATHS' in globals():
            num_images = len(IMAGE_PATHS)
            max_workers = 2 if num_images <= 4 else (4 if num_images <= 12 else 6)
        else:
            max_workers = 4
    
    return test_model_parallel(model_name, max_workers=max_workers)


print("✅ Ячейка 8: Клиент для LM Studio определен")
print("   ⚠️  НЕ запускает сервер - только отправляет запросы!")
print("   ⚠️  Контекст настраивается в GUI LM Studio")
print("\n💡 Использование:")
print('   results = test_model("olmOCR-2-7B-1025")')


✅ Ячейка 8: Клиент для LM Studio определен
   ⚠️  НЕ запускает сервер - только отправляет запросы!
   ⚠️  Контекст настраивается в GUI LM Studio

💡 Использование:
   results = test_model("olmOCR-2-7B-1025")


In [9]:
results = test_model("olmOCR-2-7B-1025")

🚀 ТЕСТИРОВАНИЕ: olmOCR-2-7B-1025
✅ LM Studio доступен на порту 1234
📊 Изображений: 12
⚡ Параллельных потоков: 4
🖼️  Размер изображений: 512px
🔄 Retry: 3 попытки

📦 Задач: 12
   ~3.0 задач на worker



⚡ olmOCR-2-7B-1025:   0%|          | 0/12 [00:00<?, ?img/s]

2025-12-22 03:06:12,520 - INFO - HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 400 Bad Request"
2025-12-22 03:06:12,521 - ERROR - ❌ page_002.png: Error code: 400 - {'error': 'Trying to keep the first 43362 tokens when context the overflows. However, the model is loaded with context length of only 32768 tokens, which is not enough. Try to load the model with a larger context length, or provide a shorter input'}
2025-12-22 03:06:12,521 - WARNING - ⚠️ page_002.png: Retry 1/3, уменьшаю размер 512px → 409px
2025-12-22 03:06:40,770 - INFO - HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-22 03:06:40,775 - INFO - ✅ page_004.png: (1211, 1713) → 512px → 35.4KB → 26,279 tok → 284 симв за 28.46с


⚡ olmOCR-2-7B-1025:   8%|▊         | 1/12 [00:28<05:13, 28.46s/img, файл=page_004.png, время=28.5с, токены=26279, символов=284, статус=✅]

2025-12-22 03:06:40,835 - INFO - HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 400 Bad Request"
2025-12-22 03:06:40,836 - ERROR - ❌ page_001.png: Error code: 400 - {'error': 'Trying to keep the first 43018 tokens when context the overflows. However, the model is loaded with context length of only 32768 tokens, which is not enough. Try to load the model with a larger context length, or provide a shorter input'}
2025-12-22 03:06:40,836 - WARNING - ⚠️ page_001.png: Retry 1/3, уменьшаю размер 512px → 409px
2025-12-22 03:06:40,899 - INFO - HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 400 Bad Request"
2025-12-22 03:06:40,900 - ERROR - ❌ page_003.png: Error code: 400 - {'error': 'Trying to keep the first 41445 tokens when context the overflows. However, the model is loaded with context length of only 32768 tokens, which is not enough. Try to load the model with a larger context length, or provide a shorter input'}
2025-12-22 03:06:40,900 - WA

...

2025-12-22 03:13:03,645 - INFO - HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-22 03:13:03,645 - INFO - ✅ page_004.png: (774, 1082) → 409px → 35.4KB → 26,464 tok → 299 симв за 80.97с
2025-12-22 03:13:03,646 - INFO - ✅ page_004.png: Успех на попытке 2


⚡ olmOCR-2-7B-1025:  92%|█████████▏| 11/12 [06:51<00:34, 34.42s/img, файл=page_004.png, время=81.0с, токены=26464, символов=299, статус=✅]

2025-12-22 03:13:03,697 - INFO - HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 400 Bad Request"
2025-12-22 03:13:03,697 - ERROR - ❌ page_005.png: Error code: 400 - {'error': 'Trying to keep the first 35593 tokens when context the overflows. However, the model is loaded with context length of only 32768 tokens, which is not enough. Try to load the model with a larger context length, or provide a shorter input'}
2025-12-22 03:13:03,698 - WARNING - ⚠️ page_005.png: Retry 2/3, уменьшаю размер 409px → 327px
2025-12-22 03:14:50,414 - INFO - HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-22 03:14:50,415 - INFO - ✅ page_005.png: (774, 1082) → 327px → 32.0KB → 27,961 tok → 6,213 симв за 105.72с
2025-12-22 03:14:50,416 - INFO - ✅ page_005.png: Успех на попытке 3


⚡ olmOCR-2-7B-1025: 100%|██████████| 12/12 [08:38<00:00, 43.17s/img, файл=page_005.png, время=105.7с, токены=27961, символов=6213, статус=✅]


💾 Сохранение результатов...

📊 СТАТИСТИКА olmOCR-2-7B-1025
  Изображений:      12
  Папок:            3
  Общее время:      953.1с (15.9 мин)
  Throughput:       0.01 изобр/с
  Среднее время:    79.42с/изобр
  Токенов:          327,659
  Символов:         16,241
  Успешно:          100.0%

📂 По папкам:
  o-predelnom-mnogomernom-raspredelenii: 4 изобр, 60.34с, 474 симв
  standartizatsiya-i-kachestvo-zhizni: 3 изобр, 101.37с, 424 симв
  Книга_Python_К_вершинам_мастерства_Рамальо_Лучано: 5 изобр, 81.52с, 2614 симв

💾 Файлы: 5
  📄 olmOCR-2-7B-1025_o-predelnom-mnogomernom-raspredelenii_20251222_031450.md
  📄 olmOCR-2-7B-1025_standartizatsiya-i-kachestvo-zhizni_20251222_031450.md
  📄 olmOCR-2-7B-1025_Книга_Python_К_вершинам_мастерства_Рамальо_Лучано_20251222_031450.md
  📄 olmOCR-2-7B-1025_results_20251222_031450.json
  📄 olmOCR-2-7B-1025_stats_20251222_031450.csv

✅ ЗАВЕРШЕНО!



In [12]:
results = test_model("qwen3-vl-4b")

🚀 ТЕСТИРОВАНИЕ: qwen3-vl-4b
✅ LM Studio доступен на порту 1234
📊 Изображений: 12
⚡ Параллельных потоков: 4
🖼️  Размер изображений: 512px
🔄 Retry: 3 попытки

📦 Задач: 12
   ~3.0 задач на worker



⚡ qwen3-vl-4b:   0%|          | 0/12 [00:00<?, ?img/s]

2025-12-23 09:56:55,613 - INFO - Retrying request to /chat/completions in 0.463495 seconds
2025-12-23 09:56:55,658 - INFO - Retrying request to /chat/completions in 0.401237 seconds
2025-12-23 09:56:55,660 - INFO - Retrying request to /chat/completions in 0.416213 seconds
2025-12-23 09:56:55,661 - INFO - Retrying request to /chat/completions in 0.480083 seconds
2025-12-23 09:59:40,920 - INFO - HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-23 09:59:40,924 - INFO - ✅ page_004.png: (1211, 1713) → 512px → 35.4KB → 30,300 tok → 6,491 симв за 345.48с


⚡ qwen3-vl-4b:   8%|▊         | 1/12 [05:45<1:03:20, 345.48s/img, файл=page_004.png, время=345.5с, токены=30300, символов=6491, статус=✅]

2025-12-23 09:59:40,994 - INFO - HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 400 Bad Request"
2025-12-23 09:59:40,995 - ERROR - ❌ page_003.png: Error code: 400 - {'error': 'Trying to keep the first 41434 tokens when context the overflows. However, the model is loaded with context length of only 33178 tokens, which is not enough. Try to load the model with a larger context length, or provide a shorter input'}
2025-12-23 09:59:40,995 - WARNING - ⚠️ page_003.png: Retry 1/3, уменьшаю размер 512px → 409px
2025-12-23 09:59:41,054 - INFO - HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 400 Bad Request"
2025-12-23 09:59:41,055 - ERROR - ❌ page_001.png: Error code: 400 - {'error': 'Trying to keep the first 43007 tokens when context the overflows. However, the model is loaded with context length of only 33178 tokens, which is not enough. Try to load the model with a larger context length, or provide a shorter input'}
2025-12-23 09:59:41,055 - WA

...

2025-12-23 10:29:41,849 - ERROR - ❌ page_004.png: Request timed out.
2025-12-23 10:29:41,851 - WARNING - ⚠️ page_004.png: Retry 2/3
2025-12-23 10:29:42,175 - INFO - Retrying request to /chat/completions in 0.975415 seconds
2025-12-23 10:32:42,980 - INFO - Retrying request to /chat/completions in 0.486897 seconds
2025-12-23 10:32:43,251 - ERROR - ❌ page_005.png: Request timed out.


⚡ qwen3-vl-4b:  92%|█████████▏| 11/12 [38:47<03:37, 217.55s/img, файл=page_005.png, время=541.8с, токены=0, символов=0, статус=❌]

2025-12-23 10:33:09,867 - INFO - HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-23 10:33:09,867 - INFO - ✅ page_004.png: (774, 1082) → 409px → 35.4KB → 26,903 tok → 2,255 симв за 207.02с
2025-12-23 10:33:09,868 - INFO - ✅ page_004.png: Успех на попытке 3


⚡ qwen3-vl-4b: 100%|██████████| 12/12 [39:14<00:00, 196.20s/img, файл=page_004.png, время=207.0с, токены=26903, символов=2255, статус=✅]


💾 Сохранение результатов...

📊 СТАТИСТИКА qwen3-vl-4b
  Изображений:      12
  Папок:            3
  Общее время:      4494.4с (74.9 мин)
  Throughput:       0.00 изобр/с
  Среднее время:    374.53с/изобр
  Токенов:          196,286
  Символов:         21,509
  Успешно:          58.3%
  ⚠️  Ошибок:       5

📂 По папкам:
  o-predelnom-mnogomernom-raspredelenii: 4 изобр, 385.69с, 2853 симв
  standartizatsiya-i-kachestvo-zhizni: 3 изобр, 461.12с, 684 симв
  Книга_Python_К_вершинам_мастерства_Рамальо_Лучано: 5 изобр, 313.64с, 1609 симв

💾 Файлы: 5
  📄 qwen3-vl-4b_o-predelnom-mnogomernom-raspredelenii_20251223_103309.md
  📄 qwen3-vl-4b_standartizatsiya-i-kachestvo-zhizni_20251223_103309.md
  📄 qwen3-vl-4b_Книга_Python_К_вершинам_мастерства_Рамальо_Лучано_20251223_103309.md
  📄 qwen3-vl-4b_results_20251223_103309.json
  📄 qwen3-vl-4b_stats_20251223_103309.csv

✅ ЗАВЕРШЕНО!

